# Derive Synonyms from Content

In [1]:
from fastcore.foundation import L
from fastcore.test import *

In [2]:
import re
import string
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer(language='english')
_re_spec = re.compile(r'([/#\\-\\.:])')

def spec_add_spaces(t):
    "Add spaces around . : - / \ and #"
    return _re_spec.sub(r' \1 ', t)

# Causes the resulting RE to match from m to n repetitions of the preceding RE, attempting to match as many repetitions as possible.
_re_space = re.compile(' {2,}')

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return _re_space.sub(' ', t)

def rm_punct(t):
    for p in string.punctuation:
        t = t.replace(p, ' ')
    return t

def transform_training_data(product_name):
    "Transform product name by replacing punctuations, removing multiple spaces, stemming, lower casing"
    name = product_name

    # replace_punct
    name = spec_add_spaces(name)
    
    # remove punct
    name = rm_punct(name)
    
    # replace multiple spaces
    name = rm_useless_spaces(name)
    
    # fix registered, trademark, copyright symbol
    # remove non-ascii characters from name
    name = name.encode(encoding='ascii', errors='ignore').decode()
    
    name = ' '.join([o for o in name.split(' ') if not o.isnumeric()])
    
    # add stemmer
    name = stemmer.stem(name)
    return name.lower()

In [3]:
def evaluate(model, k=4):
    types='headphones,televisions,refrigerator,washing machines,smartwatch'
    brands='sony,samsung,bosch,apple,skullcandy'
    models='iphone,ipad,thinkpad,iwatch,macbook'
    attributes='black,red,waterproof,wireless,bluetooth'

    evaluation=[]
    for o in [types, brands, models, attributes]: evaluation.extend(o.split(','))
    
    return [(o, model.get_nearest_neighbors(o, k=k)) for o in evaluation]

In [53]:
name='NuForce - Icon uDAC-2 USB Audio Receiver and Digital-to-Analog Converter - Silver'

In [54]:
transform_training_data('NuForce - Icon uDAC-2 USB Audio Receiver and Digital-to-Analog Converter - Silver')

'nuforce icon udac usb audio receiver and digital to analog converter silv'

In [56]:
test_str = 'iphone®™'

In [57]:
transform_training_data(test_str)

'iphon'

The utility extractTitles.py in the week3 directory goes through all of the products in the pruned XML files (i.e., everything except music and movies) and outputs their titles as plain text.

In [1]:
!python extractTitles.py

Writing results to /workspace/datasets/fasttext/titles.txt


 By default, it will save the extracted output to /workspace/datasets/fasttext/titles.txt. Also, by default, it will only sample 10% of the product titles, but you can change that with the -sample_rate parameter. 

In [55]:
!tail /workspace/datasets/fasttext/titles.txt

maytag cu ft cycle gas dryer whit
whirlpool self cleaning drop in electric oven biscuit
maytag cu ft cycle supersize capacity plus washer whit
maytag cu ft cycle gas steam dryer whit
whirlpool self cleaning drop in electric range whit
whirlpool self cleaning drop in electric oven black
maytag cu ft cycle gas dryer whit
eric clapton clapton acoustic dvd
greg koch guitar gristle dvd
pro tools le beginner level dvd


In [3]:
!/home/gitpod/fastText-0.9.2/fasttext skipgram -input /workspace/datasets/fasttext/titles.txt -output /workspace/datasets/fasttext/title_model

Read 0M words
Number of words:  2535
Number of labels: 0
Progress: 100.0% words/sec/thread:    3663 lr:  0.000000 avg.loss:  2.528054 ETA:   0h 0m 0s 0.008222 avg.loss:  2.611043 ETA:   0h 0m 2s 2.586558 ETA:   0h 0m 1s


In [4]:
!/home/gitpod/fastText-0.9.2/fasttext nn /workspace/datasets/fasttext/title_model.bin

Query word? ^C


In [5]:
import fasttext as ft

In [6]:
model=ft.train_unsupervised('/workspace/datasets/fasttext/titles.txt')

Read 0M words
Number of words:  2535
Number of labels: 0
Progress: 100.0% words/sec/thread:   55515 lr:  0.000000 avg.loss:  2.608343 ETA:   0h 0m 0s


In [10]:
L(model.words)

(#2535) ['-','</s>','for','Black','with','and','Digital','Case','Memory','/'...]

In [12]:
model.get_nearest_neighbors('iphone')

[(0.9725721478462219, 'Phones'),
 (0.9674215316772461, 'Phone'),
 (0.9665066003799438, 'Motorola'),
 (0.9517971873283386, 'Microphone'),
 (0.9450824856758118, 'iPhone'),
 (0.9434287548065186, 'No-Contract'),
 (0.9432931542396545, 'Headphones'),
 (0.9296050071716309, 'Microphones'),
 (0.9289809465408325, 'Smartphones'),
 (0.9250837564468384, 'Nokia')]

Establish a set of 20 tokens that you’ll use for evaluation. In order to get a good overall sense, you’ll want some variety: 
e.g., 5 product types (e.g., headphones), 5 brands (e.g., sony), 5 models (e.g., thinkpad) and 5 attributes (e.g., black)

types: headphones, televisions, refrigerator, washing machines, smartwatch

brands: sony, samsung, bosch, apple, skullcandy

models: iphone, ipad, thinkpad, iwatch, macbook

attributes: black, red, waterproof, wireless, bluetooth

## Extract Titles

In [27]:
import os
import random
import xml.etree.ElementTree as ET
import argparse
from pathlib import Path

directory = r'/workspace/search_with_machine_learning_course/data/pruned_products'
output_file = "/workspace/datasets/fasttext/titles.txt"

# def transform_training_data(name):
#     # IMPLEMENT
#     return name.replace('\n', ' ')

# Directory for product data
filenames = [filename for filename in os.listdir(directory) if filename.endswith(".xml")]

def process_file(filename, sample_rate=0.1):
    items = []
    print("Processing %s" % filename)
    f = os.path.join(directory, filename)
    tree = ET.parse(f)
    root = tree.getroot()
    for child in root:
        if random.random() > sample_rate:
            continue
        if (child.find('name') is not None and child.find('name').text is not None):
            name = transform_training_data(child.find('name').text)
            items.append(name)
            #output.write(name + "\n")
    return items

# print("Writing results to %s" % output_file)
# with open(output_file, 'w') as output:
#     for filename in os.listdir(directory):
#         if filename.endswith(".xml"):
#             f = os.path.join(directory, filename)
#             tree = ET.parse(f)
#             root = tree.getroot()
#             for child in root:
#                 if random.random() > sample_rate:
#                     continue
#                 if (child.find('name') is not None and child.find('name').text is not None):
#                     name = transform_training_data(child.find('name').text)
#                     output.write(name + "\n")

In [28]:
all_items = []

for each in filenames:
    all_items.extend(process_file(each))

Processing pruned_products_1.xml
Processing pruned_products_2.xml
Processing pruned_products_3.xml
Processing pruned_products_4.xml
Processing pruned_products_5.xml
Processing pruned_products_6.xml


In [29]:
len(all_items)

11358

In [30]:
L(all_items)

(#11358) ['pioneer 50w x mosfet apple® ipod® hd radio ready in dash cd deck','pantech pocket 4g mobile phone black at t ','orphen scion of sorcery playstation ps2 ','sauder audio video storage cabinet sky ald','wipeout ps vita','jura ena one touch cappuccino and latte macchiato maker silv','apple itunes gift card','toshiba satellite laptop intel® core™ i3 processor display matrix graphit','airline tycoon window','best buy gc have a ball gift card'...]

In [31]:
import nltk
nltk.download('punkt')

tokens = L(all_items).map(nltk.word_tokenize)

[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [32]:
all_tokens = []
for each in tokens: all_tokens.extend(each)

In [33]:
from collections import Counter
tok_count = Counter(all_tokens)

In [34]:
tok_count.most_common(20)

[('black', 2037),
 ('for', 1669),
 ('with', 1106),
 ('and', 995),
 ('digital', 798),
 ('case', 683),
 ('camera', 669),
 ('memory', 662),
 ('window', 617),
 ('whit', 603),
 ('card', 575),
 ('cu', 573),
 ('ft', 573),
 ('the', 563),
 ('apple®', 532),
 ('in', 514),
 ('electric', 454),
 ('nintendo', 439),
 ('laptop', 426),
 ('sony', 415)]

In [35]:
# 10 rare words
tok_count.most_common()[-10:-1]

[('satellites', 1),
 ('sparkcash', 1),
 ('ovation', 1),
 ('celebrity', 1),
 ('h550', 1),
 ('cannon', 1),
 ('position', 1),
 ('capcell', 1),
 ('positive', 1)]

In [36]:
# 20 rare words
tok_count.most_common()[-20:-1]

[('humminbird', 1),
 ('343c', 1),
 ('painter', 1),
 ('northstar', 1),
 ('blueant', 1),
 ('supertooth', 1),
 ('spectrobes', 1),
 ('portals', 1),
 ('allman', 1),
 ('recovering', 1),
 ('satellites', 1),
 ('sparkcash', 1),
 ('ovation', 1),
 ('celebrity', 1),
 ('h550', 1),
 ('cannon', 1),
 ('position', 1),
 ('capcell', 1),
 ('positive', 1)]

In [38]:
# 50 rare words
print(tok_count.most_common()[-50:-1])

[('clash', 1), ('chili', 1), ('uno', 1), ('radiant', 1), ('reckoning', 1), ('girlz', 1), ('really', 1), ('o2', 1), ('greas', 1), ('s9hd', 1), ('beaten', 1), ('nickelback', 1), ('reasons', 1), ('some', 1), ('mothership', 1), ('lips', 1), ('1950s', 1), ('pay', 1), ('scorpio', 1), ('eide', 1), ('designjet', 1), ('cradle', 1), ('linx', 1), ('trailblazer', 1), ('binoculars', 1), ('enclosures', 1), ('f150', 1), ('supercab', 1), ('hatchbacks', 1), ('sale', 1), ('humminbird', 1), ('343c', 1), ('painter', 1), ('northstar', 1), ('blueant', 1), ('supertooth', 1), ('spectrobes', 1), ('portals', 1), ('allman', 1), ('recovering', 1), ('satellites', 1), ('sparkcash', 1), ('ovation', 1), ('celebrity', 1), ('h550', 1), ('cannon', 1), ('position', 1), ('capcell', 1), ('positive', 1)]


## Sample Rate 0.1

In [39]:
!python extractTitles.py

Writing results to /workspace/datasets/fasttext/titles.txt


In [40]:
import fasttext as ft
model=ft.train_unsupervised('/workspace/datasets/fasttext/titles.txt')

Read 0M words
Number of words:  2348
Number of labels: 0
Progress: 100.0% words/sec/thread:   30747 lr:  0.000000 avg.loss:  2.642882 ETA:   0h 0m 0s100.0% words/sec/thread:   30752 lr: -0.000011 avg.loss:  2.642882 ETA:   0h 0m 0s


In [41]:
print(model.words[:50])

['</s>', 'black', 'for', 'with', 'and', 'digital', 'camera', 'case', 'memory', 'window', 'whit', 'card', 'cu', 'ft', 'the', 'in', 'apple®', 'nintendo', 'electric', 'laptop', 'sony', 'wireless', 'series', 'steel', 'stainless', 'playstation', 'guitar', 'hard', 'ge', 'refrigerator', 'system', 'pack', 'side', 'mobile', 's', 'samsung', 'extra', 'range', 'kit', 'on', 'game', 'battery', 'usb', 'bag', 'select', 'd', 'built', 'silv', 'xbox', 'r']


In [42]:
model.get_nearest_neighbors('iphone')

[(0.9884212613105774, 'phones'),
 (0.987588107585907, 'iphone®'),
 (0.9804374575614929, 'gophone'),
 (0.9657481908798218, 'phone'),
 (0.9601891040802002, 'smartphones'),
 (0.9589815139770508, 'mobile'),
 (0.9587773084640503, 'headphone'),
 (0.9553311467170715, 'phon'),
 (0.947689950466156, 'microphone'),
 (0.9473929405212402, 'microphones')]

In [7]:
def evaluate(model, k=4):
    types='headphones,televisions,refrigerator,washing machines,smartwatch'
    brands='sony,samsung,bosch,apple,skullcandy'
    models='iphone,ipad,thinkpad,iwatch,macbook'
    attributes='black,red,waterproof,wireless,bluetooth'

    evaluation=[]
    for o in [types, brands, models, attributes]: evaluation.extend(o.split(','))
    
    return [(o, model.get_nearest_neighbors(o, k=k)) for o in evaluation]

In [43]:
types='headphones,televisions,refrigerator,washing machines,smartwatch'
brands='sony,samsung,bosch,apple,skullcandy'
models='iphone,ipad,thinkpad,iwatch,macbook'
attributes='black,red,waterproof,wireless,bluetooth'
evaluation=[]
for o in [types, brands, models, attributes]: evaluation.extend(o.split(','))

In [44]:
[(o, model.get_nearest_neighbors(o, k=4)) for o in evaluation]

[('headphones',
  [(0.9919744729995728, 'headphone'),
   (0.9893032908439636, 'microphones'),
   (0.987011194229126, 'headphon'),
   (0.9685732126235962, 'microphone')]),
 ('televisions',
  [(0.9955301284790039, 'dolphins'),
   (0.99502032995224, 'anaheim'),
   (0.9947879314422607, 'ibm'),
   (0.9944508075714111, 'battlefield')]),
 ('refrigerator',
  [(0.9975049495697021, 'by'),
   (0.9957539439201355, 'thru'),
   (0.9954434037208557, 'refriger'),
   (0.993172287940979, 'side')]),
 ('washing machines',
  [(0.9951592087745667, 'maple'),
   (0.9936034679412842, 'vertical'),
   (0.9935685992240906, 'washington'),
   (0.9935431480407715, 'marshmallow')]),
 ('smartwatch',
  [(0.9890354871749878, 'fat'),
   (0.98868328332901, 'bravo'),
   (0.9883300065994263, 'house'),
   (0.9879694581031799, 'patch')]),
 ('sony',
  [(0.9773331880569458, 'dcr'),
   (0.9768596291542053, 'everio'),
   (0.9755401611328125, 'pink'),
   (0.9712700247764587, 'kodak')]),
 ('samsung',
  [(0.9749250411987305, 'sharp'

## Sample Rate 1.0

In [58]:
!python extractTitles.py --sample_rate 1.0

Writing results to /workspace/datasets/fasttext/titles.txt


In [59]:
!wc -l /workspace/datasets/fasttext/titles.txt

115358 /workspace/datasets/fasttext/titles.txt


In [4]:
import fasttext as ft
model=ft.train_unsupervised('/workspace/datasets/fasttext/titles.txt', lr=0.1, epoch=25)

Read 1M words
Number of words:  8651
Number of labels: 0
Progress: 100.0% words/sec/thread:   52372 lr:  0.000000 avg.loss:  1.004333 ETA:   0h 0m 0s


In [65]:
model.lr, model.epoch, model.minCount, model.wordNgrams

(0.05, 5, 5, 1)

In [49]:
ft.train_unsupervised??

Signature: ft.train_unsupervised(*kargs, **kwargs)
Source:   
def train_unsupervised(*kargs, **kwargs):
    """
    Train an unsupervised model and return a model object.

    input must be a filepath. The input text does not need to be tokenized
    as per the tokenize function, but it must be preprocessed and encoded
    as UTF-8. You might want to consult standard preprocessing scripts such
    as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html

    The input field must not contain any labels or use the specified label prefix
    unless it is ok for those words to be ignored. For an example consult the
    dataset pulled by the example script word-vector-example.sh, which is
    part of the fastText repository.
    """
    arg_names = ['input', 'model', 'lr', 'dim', 'ws', 'epoch', 'minCount',
                 'minCountLabel', 'minn', 'maxn', 'neg', 'wordNgrams', 'loss', 'bucket',
                 'thread', 'lrUpdateRate', 't', 'label', 'verbose', 'pretrained

In [5]:
evaluate(model, k=5)

[('headphones',
  [(0.8895946741104126, 'earbud'),
   (0.8213903307914734, 'headphon'),
   (0.8014456033706665, 'ear'),
   (0.7627490162849426, 'headphone'),
   (0.7085865139961243, 'uprock')]),
 ('televisions',
  [(0.9032291769981384, 'television'),
   (0.8672158122062683, 'televis'),
   (0.6146353483200073, 'purevision'),
   (0.5986377596855164, 'sunbritetv'),
   (0.5626944303512573, 'tvs')]),
 ('refrigerator',
  [(0.8376485109329224, 'side'),
   (0.8363350033760071, 'refrigerators'),
   (0.7642907500267029, 'ft'),
   (0.7570213675498962, 'cu'),
   (0.7307917475700378, 'ultrafinish')]),
 ('washing machines',
  [(0.7319537997245789, 'machines'),
   (0.6987502574920654, 'washing'),
   (0.6780431866645813, 'orioles'),
   (0.6701635718345642, 'emachines'),
   (0.6558340191841125, 'ravens')]),
 ('smartwatch',
  [(0.7180104851722717, 'watch'),
   (0.6671700477600098, 'smarter'),
   (0.6441442966461182, 'smartridg'),
   (0.6416373252868652, 'veteran'),
   (0.6404958963394165, 'smartstart')]

In [9]:
model10=ft.train_unsupervised('/workspace/datasets/fasttext/titles.txt',lr=0.1, epoch=25, minCount=10)

evaluate(model10, k=5)

Read 1M words
Number of words:  5743
Number of labels: 0
Progress: 100.0% words/sec/thread:   54597 lr:  0.000000 avg.loss:  1.059682 ETA:   0h 0m 0s


[('headphones',
  [(0.8930500745773315, 'earbud'),
   (0.827166736125946, 'headphon'),
   (0.7981834411621094, 'ear'),
   (0.7742078900337219, 'headphone'),
   (0.6548944115638733, 'bud')]),
 ('televisions',
  [(0.8712342381477356, 'television'),
   (0.5790674686431885, 'tvs'),
   (0.5442847013473511, 'visionmount'),
   (0.5345789790153503, 'vcr'),
   (0.5310589075088501, 'pole')]),
 ('refrigerator',
  [(0.8360533714294434, 'refrigerators'),
   (0.8313722610473633, 'side'),
   (0.7770050764083862, 'cu'),
   (0.7649127244949341, 'ft'),
   (0.7276114821434021, 'satina')]),
 ('washing machines',
  [(0.7467196583747864, 'washington'),
   (0.7333082556724548, 'emachines'),
   (0.7068591117858887, 'machines'),
   (0.6511589288711548, 'buccaneers'),
   (0.6405819654464722, 'seattle')]),
 ('smartwatch',
  [(0.7764878869056702, 'watch'),
   (0.6903185248374939, 'schedule'),
   (0.6675777435302734, 'smarter'),
   (0.6601588726043701, 'smartridge'),
   (0.6594529747962952, 'veteran')]),
 ('sony',

In [10]:
model20=ft.train_unsupervised('/workspace/datasets/fasttext/titles.txt', lr=0.1, epoch=25, minCount=20)

evaluate(model20, k=5)

Read 1M words
Number of words:  3749
Number of labels: 0
Progress: 100.0% words/sec/thread:   55466 lr:  0.000000 avg.loss:  1.141127 ETA:   0h 0m 0s100.0% words/sec/thread:   55466 lr: -0.000002 avg.loss:  1.141127 ETA:   0h 0m 0s


[('headphones',
  [(0.897201657295227, 'earbud'),
   (0.8415265679359436, 'headphon'),
   (0.8182222843170166, 'ear'),
   (0.7751890420913696, 'headphone'),
   (0.658271312713623, 'lowrider')]),
 ('televisions',
  [(0.6713113188743591, 'redskins'),
   (0.6457560658454895, 'dolphins'),
   (0.6449776887893677, 'panthers'),
   (0.6283527612686157, 'bengals'),
   (0.6211680173873901, 'titans')]),
 ('refrigerator',
  [(0.8227670192718506, 'side'),
   (0.7185763716697693, 'cu'),
   (0.7053077816963196, 'ft'),
   (0.6986026763916016, 'satina'),
   (0.6966159343719482, 'refriger')]),
 ('washing machines',
  [(0.7679130434989929, 'washington'),
   (0.7256389856338501, 'emachines'),
   (0.6501688957214355, 'seattle'),
   (0.6486727595329285, 'houston'),
   (0.6449505090713501, 'oakland')]),
 ('smartwatch',
  [(0.7220542430877686, 'watch'),
   (0.6277055740356445, 'tailgate'),
   (0.6095103025436401, 'smartridge'),
   (0.6089803576469421, 'schedule'),
   (0.6019293069839478, 'veteran')]),
 ('sony

In [11]:
model50=ft.train_unsupervised('/workspace/datasets/fasttext/titles.txt', lr=0.1, epoch=25, minCount=50)
evaluate(model50, k=5)

Read 1M words
Number of words:  2062
Number of labels: 0
Progress: 100.0% words/sec/thread:   64948 lr:  0.000000 avg.loss:  1.191281 ETA:   0h 0m 0s20s


[('headphones',
  [(0.8592641949653625, 'earbud'),
   (0.820675253868103, 'headphon'),
   (0.7921386361122131, 'ear'),
   (0.6619364619255066, 'skullcandy'),
   (0.5847864747047424, 'bud')]),
 ('televisions',
  [(0.6880020499229431, 'lions'),
   (0.6864220499992371, 'panthers'),
   (0.6717005968093872, 'colts'),
   (0.6708186268806458, 'ravens'),
   (0.6689649820327759, 'dolphins')]),
 ('refrigerator',
  [(0.7834730744361877, 'side'),
   (0.7447421550750732, 'cu'),
   (0.7315928936004639, 'ft'),
   (0.7196537852287292, 'satina'),
   (0.702346682548523, 'monochromatic')]),
 ('washing machines',
  [(0.7412802577018738, 'emachines'),
   (0.7162814736366272, 'washington'),
   (0.6460802555084229, 'kansas'),
   (0.621627151966095, 'chiefs'),
   (0.6208103895187378, 'arizona')]),
 ('smartwatch',
  [(0.7734754681587219, 'watch'),
   (0.6213749051094055, 'smart'),
   (0.5565332770347595, 'schedule'),
   (0.5382906198501587, 'men'),
   (0.5303767323493958, 'heart')]),
 ('sony',
  [(0.5657361149